In [1]:
# Imports used across ETL exploration and final incremental pipeline
import time
import os, json, shutil
from datetime import datetime, timezone
from pyspark.sql.functions import col, current_timestamp, input_file_name, to_date, unix_timestamp, broadcast, desc, when, lit, concat, floor, rand, regexp_replace, sum as Fsum
from pyspark.sql import SparkSession

In [2]:
# Centralized project paths for input, output, lookup, and ETL state
BASE_PATH = "/home/jovyan/work"

INBOX = f"{BASE_PATH}/data/inbox"
OUTBOX = f"{BASE_PATH}/data/outbox/trips_enriched.parquet"
LOOKUP = f"{BASE_PATH}/data/taxi_zone_lookup.parquet"
STATE = f"{BASE_PATH}/state/manifest.json"

In [3]:
# Read manifest file that tracks already processed input files
def load_manifest(path):
    if not os.path.exists(path):
        return {"processed_files": []}
    with open(path, "r") as f:
        return json.load(f)

In [4]:
# Persist updated manifest to keep ingestion incremental and idempotent
def save_manifest(path, manifest):
    with open(path, "w") as f:
        json.dump(manifest, f, indent=2)

In [5]:
# Compute full input list and the subset of files that are new in this run
manifest = load_manifest(STATE)
processed = set(x["filename"] for x in manifest["processed_files"])

all_files = sorted([f for f in os.listdir(INBOX) if f.endswith(".parquet")])
new_files = [f for f in all_files if f not in processed]

all_files, new_files

(['yellow_tripdata_2025-01.parquet', 'yellow_tripdata_2025-02.parquet'],
 ['yellow_tripdata_2025-01.parquet', 'yellow_tripdata_2025-02.parquet'])

In [6]:
# Initialize Spark session for the ETL run
spark = SparkSession.builder.appName("Project1-ETL").getOrCreate()

## ETL rules used in the final run

This cell documents the business rules implemented in the final incremental Spark job.

- **Cleaning rules**: keep only rows where required fields are non-null, `passenger_count > 0`, `trip_distance > 0`, and `tpep_dropoff_datetime > tpep_pickup_datetime`.
- **Dedup key**: `VendorID`, `tpep_pickup_datetime`, `tpep_dropoff_datetime`, `PULocationID`, `DOLocationID`.
- **Incremental behavior**: process only files not present in `state/manifest.json`.
- **Idempotency**: anti-join new batch against existing outbox using the dedup key, then append only unseen records.

In [7]:
# Helper to verify whether output parquet folder already contains data files
def outbox_has_data(path: str) -> bool:
    return (
        os.path.exists(path)
        and os.path.isdir(path)
        and any(name.startswith("part-") and name.endswith(".parquet") for name in os.listdir(path))
    )

In [8]:
# Cleanup safety: remove empty/corrupted outbox folder before writing
if os.path.exists(OUTBOX) and not outbox_has_data(OUTBOX):
    print("OUTBOX exists but has no parquet parts. Deleting corrupted/empty outbox folder...")
    shutil.rmtree(OUTBOX)

In [9]:
# Log file discovery result for this run (all vs new files)
print("All files:", all_files)
print("New files:", new_files)

All files: ['yellow_tripdata_2025-01.parquet', 'yellow_tripdata_2025-02.parquet']
New files: ['yellow_tripdata_2025-01.parquet', 'yellow_tripdata_2025-02.parquet']


In [10]:
# Final incremental ETL: process only new files, clean, dedup, enrich, append, and update manifest
if not new_files:
    print("Nothing new to process. Exiting without changes.")
else:
    start = time.time()
    now = datetime.now(timezone.utc).isoformat()

    new_paths = [os.path.join(INBOX, f) for f in new_files]
    df_new = spark.read.parquet(*new_paths).withColumn("source_file", input_file_name())

    # ---------- Types ----------
    df_typed = (
        df_new
        .withColumn("tpep_pickup_datetime", col("tpep_pickup_datetime").cast("timestamp"))
        .withColumn("tpep_dropoff_datetime", col("tpep_dropoff_datetime").cast("timestamp"))
        .withColumn("passenger_count", col("passenger_count").cast("int"))
        .withColumn("trip_distance", col("trip_distance").cast("double"))
        .withColumn("PULocationID", col("PULocationID").cast("int"))
        .withColumn("DOLocationID", col("DOLocationID").cast("int"))
        .withColumn("VendorID", col("VendorID").cast("int"))
    )

    # ---------- Cleaning ----------
    df_clean = df_typed.filter(
        col("tpep_pickup_datetime").isNotNull()
        & col("tpep_dropoff_datetime").isNotNull()
        & col("passenger_count").isNotNull()
        & col("trip_distance").isNotNull()
        & col("PULocationID").isNotNull()
        & col("DOLocationID").isNotNull()
        & col("VendorID").isNotNull()
        & (col("passenger_count") > 0)
        & (col("trip_distance") > 0)
        & (col("tpep_dropoff_datetime") > col("tpep_pickup_datetime"))
    )

    # ---------- Dedup ----------
    dedup_key = ["VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", "PULocationID", "DOLocationID"]

    # 1) Remove duplicates within the current batch
    df_dedup_batch = df_clean.dropDuplicates(dedup_key)

   # 2) Optionally remove records that already exist in OUTBOX (ensures idempotency even without relying solely on the manifest)
    if outbox_has_data(OUTBOX):
        existing_keys = (
            spark.read.parquet(OUTBOX)
            .select(*dedup_key)
            .dropDuplicates(dedup_key)
        )
        df_dedup = df_dedup_batch.join(existing_keys, on=dedup_key, how="left_anti")
    else:
        df_dedup = df_dedup_batch

    # ---------- Enrichment ----------
    zones = spark.read.parquet(LOOKUP)

    zones_pickup = zones.select(
        col("LocationID").alias("PULocationID"),
        col("Zone").alias("pickup_zone")
    )

    zones_dropoff = zones.select(
        col("LocationID").alias("DOLocationID"),
        col("Zone").alias("dropoff_zone")
    )

    df_enriched = (
        df_dedup
        .join(broadcast(zones_pickup), on="PULocationID", how="left")
        .join(broadcast(zones_dropoff), on="DOLocationID", how="left")
    )

    # ---------- Derived columns ----------
    df_ready = (
        df_enriched
        .withColumn(
            "trip_duration_minutes",
            (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60.0
        )
        .withColumn("pickup_date", to_date("tpep_pickup_datetime"))
        .withColumn("ingested_at", current_timestamp())
    )

    # ---------- Select required output columns (helps grading) ----------
    df_out = df_ready.select(
        "tpep_pickup_datetime", "tpep_dropoff_datetime",
        "PULocationID", "DOLocationID",
        "pickup_zone", "dropoff_zone",
        "passenger_count", "trip_distance",
        "trip_duration_minutes", "pickup_date",
        "source_file", "ingested_at"
    )

    # ---------- Metrics for README ----------
    input_rows = df_new.count()
    clean_rows = df_clean.count()
    dedup_rows = df_dedup.count()
    final_rows = df_out.count()

    print("Row counts (new batch):", {
        "input": input_rows,
        "after_cleaning": clean_rows,
        "after_dedup": dedup_rows,
        "final_output": final_rows
    })

    # ---------- Write ----------
    if final_rows > 0:
        df_out.coalesce(4).write.mode("append").parquet(OUTBOX)
        wrote = final_rows
    else:
        wrote = 0

    # ---------- Runtime ----------
    end = time.time()
    print("Full job runtime (seconds):", round(end - start, 2))
    print("Rows written:", wrote)

    # ---------- Manifest update (always) ----------
    for f in new_files:
        file_path = os.path.join(INBOX, f)
        manifest["processed_files"].append({
            "filename": f,
            "size_bytes": os.path.getsize(file_path),
            "processed_at": now
        })

    save_manifest(STATE, manifest)

    print("Run complete!")
    print("Output path:", OUTBOX)
    print("Manifest updated:", STATE)
    print("Cleaning rules: non-null required fields, passenger_count > 0, trip_distance > 0, dropoff > pickup")
    print("Dedup key:", dedup_key)

Row counts (new batch): {'input': 7052769, 'after_cleaning': 5579927, 'after_dedup': 5486133, 'final_output': 5486133}
Full job runtime (seconds): 35.77
Rows written: 5486133
✅ Run complete!
Output path: /home/jovyan/work/data/outbox/trips_enriched.parquet
Manifest updated: /home/jovyan/work/state/manifest.json
Cleaning rules: non-null required fields, passenger_count > 0, trip_distance > 0, dropoff > pickup
Dedup key: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID']


# Bad rows Examples

In [13]:
# Trip distance <= 0
bad_rows = df_typed.filter(
    (col("trip_distance") <= 0)
)

bad_rows.select(
    "passenger_count",
    "trip_distance",
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime"
).show(3, truncate=False)

+---------------+-------------+--------------------+---------------------+
|passenger_count|trip_distance|tpep_pickup_datetime|tpep_dropoff_datetime|
+---------------+-------------+--------------------+---------------------+
|1              |0.0          |2025-01-01 00:49:48 |2025-01-01 00:49:48  |
|1              |0.0          |2025-01-01 00:37:43 |2025-01-01 00:37:53  |
|3              |0.0          |2025-01-01 00:57:08 |2025-01-01 00:57:16  |
+---------------+-------------+--------------------+---------------------+
only showing top 3 rows



In [14]:
# Dropoff time < Pickup time

bad_rows = df_typed.filter(
    (col("tpep_dropoff_datetime") < col("tpep_pickup_datetime"))
)

bad_rows.select(
    "passenger_count",
    "trip_distance",
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime"
).show(3, truncate=False)

+---------------+-------------+--------------------+---------------------+
|passenger_count|trip_distance|tpep_pickup_datetime|tpep_dropoff_datetime|
+---------------+-------------+--------------------+---------------------+
|1              |9.0          |2025-01-02 12:26:00 |2025-01-02 11:29:58  |
|1              |3.8          |2025-01-06 16:00:00 |2025-01-06 15:05:30  |
|1              |1.0          |2025-01-15 15:00:00 |2025-01-15 14:42:48  |
+---------------+-------------+--------------------+---------------------+
only showing top 3 rows



In [16]:
# Passenger Count <= 0

bad_rows = df_typed.filter(
    (col("passenger_count") <= 0)
)

bad_rows.select(
    "passenger_count",
    "trip_distance",
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime"
).show(3, truncate=False)


+---------------+-------------+--------------------+---------------------+
|passenger_count|trip_distance|tpep_pickup_datetime|tpep_dropoff_datetime|
+---------------+-------------+--------------------+---------------------+
|0              |0.4          |2025-01-01 00:14:47 |2025-01-01 00:16:15  |
|0              |1.6          |2025-01-01 00:39:27 |2025-01-01 00:51:51  |
|0              |2.8          |2025-01-01 00:53:43 |2025-01-01 01:13:23  |
+---------------+-------------+--------------------+---------------------+
only showing top 3 rows



# Final Dataset

In [17]:
df_final = spark.read.parquet(OUTBOX)
df_final.show(5, truncate=False)

+--------------------+---------------------+------------+------------+-------------------------+--------------+---------------+-------------+---------------------+-----------+-------------------------------------------------------------------+--------------------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|DOLocationID|pickup_zone              |dropoff_zone  |passenger_count|trip_distance|trip_duration_minutes|pickup_date|source_file                                                        |ingested_at               |
+--------------------+---------------------+------------+------------+-------------------------+--------------+---------------+-------------+---------------------+-----------+-------------------------------------------------------------------+--------------------------+
|2025-01-01 03:38:57 |2025-01-01 04:05:11  |230         |1           |Times Sq/Theatre District|Newark Airport|4              |17.91        |26.233333333333334   |2025-01-01 |file:///home

## Scenario

In [18]:
# Identifying most frequent zone

df_scn = spark.read.parquet(OUTBOX)

zone_counts = df_scn.groupBy("pickup_zone").count()

total = df_scn.count()

top1 = (
    zone_counts
    .orderBy(desc("count"))
    .first()
)

hot_zone = top1["pickup_zone"]
hot_count = top1["count"]

print("Hot zone:", hot_zone)
print("Hot zone count:", hot_count)
print("Hot zone %:", round(100 * hot_count / total, 2), "%")

Hot zone: Upper East Side South
Hot zone count: 286719
Hot zone %: 5.23 %


## Baseline

In [19]:
# This triggers a shuffle (Exchange) because groupBy is a wide transformation.
# Use Spark UI to capture baseline metrics (duration, shuffle write/read, spill).
baseline_agg = df_scn.groupBy("pickup_zone").count()

In [22]:
# Action -> creates the baseline Spark job
# (Run this and take Spark UI screenshots for "scenario_baseline_job.png" and "scenario_baseline_stage.png")
baseline_agg.orderBy(desc("count")).show(10, truncate=False)

+----------------------------+------+
|pickup_zone                 |count |
+----------------------------+------+
|Upper East Side South       |286719|
|Midtown Center              |283351|
|Upper East Side North       |263608|
|JFK Airport                 |250019|
|Penn Station/Madison Sq West|208572|
|Midtown East                |204407|
|Times Sq/Theatre District   |202414|
|Lincoln Square East         |184968|
|LaGuardia Airport           |166738|
|Midtown North               |165699|
+----------------------------+------+
only showing top 10 rows



## Without Most Popular Zone

In [23]:
# This checks whether removing the most frequent key changes runtime / shuffle behavior.
df_no_hot = df_scn.filter(col("pickup_zone") != hot_zone)

no_hot_agg = df_no_hot.groupBy("pickup_zone").count()

# Action -> creates a Spark job (take UI screenshots for "scenario_wfz_job.png" and "scenario_wfz_stage.png")
no_hot_agg.orderBy(desc("count")).show(10, truncate=False)

+----------------------------+------+
|pickup_zone                 |count |
+----------------------------+------+
|Midtown Center              |283351|
|Upper East Side North       |263608|
|JFK Airport                 |250019|
|Penn Station/Madison Sq West|208572|
|Midtown East                |204407|
|Times Sq/Theatre District   |202414|
|Lincoln Square East         |184968|
|LaGuardia Airport           |166738|
|Midtown North               |165699|
|Union Sq                    |156884|
+----------------------------+------+
only showing top 10 rows



## Repartition

In [30]:
# repartition forces an extra shuffle.
# It can help in severe skew cases (very dominant keys) by improving distribution,
# but it may add overhead when skew is mild.
repart_agg = (
    df_scn
    .repartition("pickup_zone")      # Extra shuffle to redistribute rows by key
    .groupBy("pickup_zone")
    .count()
)

# Action -> creates a Spark job (take UI screenshots for "scenario_repart_job.png" and "scenario_repart_stage.png")
repart_agg.orderBy(desc("count")).show(10, truncate=False)

+----------------------------+------+
|pickup_zone                 |count |
+----------------------------+------+
|Upper East Side South       |286719|
|Midtown Center              |283351|
|Upper East Side North       |263608|
|JFK Airport                 |250019|
|Penn Station/Madison Sq West|208572|
|Midtown East                |204407|
|Times Sq/Theatre District   |202414|
|Lincoln Square East         |184968|
|LaGuardia Airport           |166738|
|Midtown North               |165699|
+----------------------------+------+
only showing top 10 rows

